<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b30f416190657ad722427b2addc347213b6941e42fd0a9da20f7c609dfbcff19
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-22 15:04:19
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.11 L
Current:  1.42 C
-------------------
Today PnL: -1.28 L (-0.89%)
Current PnL: -17.96 L (-12.08%)
CY Booked + Current PnL: -6.52 L (-4.39%)
-------------------
Total profit:  1.74 L
Total loss:  -19.70 L
-------------------
Total Booked + Current PnL: 20.46 L (16.81%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.05%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.26 C
Est FTT PnL: 83.48 L (58.73%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199670.0,35000.0,8326.0,1.68,21.25,4.17,26.31,89.0,4.20,1.44,30.93,X40N,NTT,AC
76,TTKPRESTIG,770.00,99.79,46.0,M-SC,3.33,86047.0,-14730.0,14826.0,-1.12,-14.62,17.23,0.09,245.0,-0.99,0.62,13.05,OX40N,NTT,DURABLES
49,NATIONALUM,244.55,-44.49,63.0,H-MC,8.54,111496.0,10992.0,18118.0,-0.58,10.94,16.25,28.96,79.0,0.61,0.80,48.62,MH,ATH,METALS
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198974.0,-1164.0,22046.0,-0.91,-0.58,11.08,10.44,4.0,-0.05,1.43,4.95,X40,NTT,FMCG
56,RELIANCE,1533.00,-13.29,49.0,H-LC,4.80,216949.0,6583.0,22194.0,-1.19,3.13,10.23,13.68,37.0,0.30,1.56,20.17,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1255.66,80.0,M-SC,6.41,184435.0,3292.0,42180.0,10.63,1.82,22.87,25.10,235.0,0.08,1.33,47.81,XY24,NTT,HEALTHCARE
8,AWL,485.00,-63.36,65.0,H-MC,3.91,260715.0,-41036.0,202471.0,6.25,-13.60,77.66,53.50,116.0,-0.20,1.88,15.80,XY24,NTT,FMCG
80,VAIBHAVGBL,521.00,63.64,66.0,H-SC,6.52,141857.0,-40918.0,151986.0,1.69,-22.39,107.14,60.77,125.0,-0.27,1.02,27.06,XR,NTT,JEWELLERY
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199670.0,35000.0,8326.0,1.68,21.25,4.17,26.31,89.0,4.20,1.44,30.93,X40N,NTT,AC
31,HINDZINC,730.22,22.76,62.0,M-LC,1.67,199774.0,-5302.0,117867.0,1.37,-2.59,59.00,54.89,52.0,-0.04,1.44,20.89,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.62,49.0,H-LC,1.42,236790.0,-9361.0,88559.0,-4.49,-3.80,37.40,32.18,16.0,-0.11,1.70,30.87,X200,ATH,IT
53,RAJESHEXPO,518.0,1795.20,53.0,L-SC,2.44,51709.0,-85468.0,85558.0,-3.65,-62.30,165.46,0.07,267.0,-1.00,0.37,28.60,OX40N,NTT,JEWELLERY
66,SONACOMS,1006.0,-40.79,38.0,M-SC,11.75,73701.0,-27458.0,102349.0,-3.45,-27.14,138.87,74.03,202.0,-0.27,0.53,3.93,AR,BTT,AUTO
83,VOLTAS,1530.0,-1.64,47.0,H-MC,2.74,206265.0,14523.0,23225.0,-3.16,7.57,11.26,19.69,99.0,0.63,1.48,14.89,XY25,NTT,AC
26,GREENPANEL,537.0,223.64,55.0,M-SC,1.26,144570.0,-38508.0,117492.0,-3.06,-21.03,81.27,43.14,230.0,-0.33,1.04,35.99,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-30.62,50.0,H-SC,16.84,198393.0,-1561.0,98780.0,-1.90,-0.78,49.79,48.63,115.0,-0.02,1.43,10.68,XR,NTT,MISC
29,HAVELLS,2069.16,-0.37,55.0,H-MC,1.91,247010.0,-721.0,75783.0,-0.47,-0.29,30.68,30.30,92.0,-0.01,1.78,12.96,X40,ATH,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.30,121770.0,-20709.0,40233.0,-1.51,-14.53,33.04,13.70,163.0,-0.51,0.88,43.72,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,99.79,46.0,M-SC,3.33,86047.0,-14730.0,14826.0,-1.12,-14.62,17.23,0.09,245.0,-0.99,0.62,13.05,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.06,63.0,H-SC,0.81,227106.0,-42561.0,78897.0,0.19,-15.78,34.74,13.47,138.0,-0.54,1.63,14.87,XY24,NTT,PAINTS
71,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.44,108728.0,-23107.0,65335.0,-0.19,-17.53,60.09,32.03,98.0,-0.35,0.78,21.78,OX40N,NTT,IT
18,CERA,9475.0,-21.00,41.0,H-SC,2.13,144072.0,-31831.0,73851.0,-1.32,-18.10,51.26,23.89,149.0,-0.43,1.04,24.43,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.6,69.0,H-MC,3.52,199670.0,35000.0,8326.0,1.68,21.25,4.17,26.31,89.0,4.2,1.44,30.93,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1255.66,80.0,M-SC,6.41,184435.0,3292.0,42180.0,10.63,1.82,22.87,25.10,235.0,0.08,1.33,47.81,XY24,NTT,HEALTHCARE
59,SAIL,228.00,47.55,68.0,M-MC,8.17,233452.0,8490.0,159168.0,-0.46,3.77,68.18,74.53,192.0,0.05,1.68,37.24,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.60,69.0,H-MC,3.52,199670.0,35000.0,8326.0,1.68,21.25,4.17,26.31,89.0,4.20,1.44,30.93,X40N,NTT,AC
17,CAMS,5211.76,-1.17,59.0,H-SC,3.11,112336.0,10332.0,33588.0,-2.17,10.13,29.90,43.06,122.0,0.31,0.81,28.98,X40N,ATH,MISC
83,VOLTAS,1530.00,-1.64,47.0,H-MC,2.74,206265.0,14523.0,23225.0,-3.16,7.57,11.26,19.69,99.0,0.63,1.48,14.89,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-44.49,63.0,H-MC,8.54,111496.0,10992.0,18118.0,-0.58,10.94,16.25,28.96,79.0,0.61,0.80,48.62,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-41.04,49.0,M-SC,0.82,99750.0,8252.0,70503.0,-0.58,9.02,70.68,86.07,223.0,0.12,0.72,44.85,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.60,185529.0,5721.0,106698.0,0.13,3.18,57.51,62.52,88.0,0.05,1.33,38.35,XR,NTT,BANKS
36,INDIAMART,4850.92,-53.52,32.0,H-SC,11.35,129808.0,6472.0,127290.0,-0.83,5.25,98.06,108.45,119.0,0.05,0.93,28.77,AR,ATH,MISC
85,WIPRO,420.00,-11.52,49.0,M-LC,5.68,155223.0,4278.0,105179.0,-2.16,2.83,67.76,72.51,53.0,0.04,1.12,9.03,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.93,37.0,H-LC,6.88,295713.0,-20129.0,132805.0,-2.31,-6.37,44.91,35.67,5.0,-0.15,2.13,6.60,X40N,ATH,FMCG
73,TCS,4476.75,-27.40,44.0,H-LC,12.24,288928.0,-57028.0,131896.0,-3.01,-16.48,45.65,21.64,1.0,-0.43,2.08,2.35,X40,ATH,IT
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.73,218539.0,-33229.0,86432.0,0.07,-13.20,39.55,21.13,27.0,-0.38,1.57,17.55,X40,ATH,PAINTS
19,CIPLA,1795.00,-20.81,45.0,H-LC,5.49,211336.0,6836.0,34575.0,-2.11,3.34,16.36,20.25,10.0,0.20,1.52,12.37,X40N,BTT,PHARMA
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198974.0,-1164.0,22046.0,-0.91,-0.58,11.08,10.44,4.0,-0.05,1.43,4.95,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-4.62,49.0,H-LC,1.42,236790.0,-9361.0,88559.0,-4.49,-3.80,37.40,32.18,16.0,-0.11,1.70,30.87,X200,ATH,IT
41,ITC,452.0,-38.34,45.0,H-LC,2.28,198974.0,-1164.0,22046.0,-0.91,-0.58,11.08,10.44,4.0,-0.05,1.43,4.95,X40,NTT,FMCG
64,SIEMENS,4671.5,1.03,55.0,H-LC,3.59,162250.0,-23845.0,71325.0,-2.43,-12.81,43.96,25.51,15.0,-0.33,1.17,19.57,AR,ATH,ELECTRICAL
50,NESTLEIND,1377.0,-9.75,50.0,H-LC,4.24,277875.0,12449.0,44349.0,-0.59,4.69,15.96,21.40,11.0,0.28,2.00,12.36,XY25,NTT,FMCG
56,RELIANCE,1533.0,-13.29,49.0,H-LC,4.80,216949.0,6583.0,22194.0,-1.19,3.13,10.23,13.68,37.0,0.30,1.56,20.17,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.40,44.0,H-LC,12.24,288928.0,-57028.0,131896.0,-3.01,-16.48,45.65,21.64,1.0,-0.43,2.08,2.35,X40,ATH,IT
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198974.0,-1164.0,22046.0,-0.91,-0.58,11.08,10.44,4.0,-0.05,1.43,4.95,X40,NTT,FMCG
79,UNITDSPR,1644.00,-11.95,56.0,H-LC,7.66,230721.0,-3423.0,52051.0,0.99,-1.46,22.56,20.77,86.0,-0.07,1.66,5.35,X40N,NTT,BREWERIES
82,VBL,671.64,-17.93,37.0,H-LC,6.88,295713.0,-20129.0,132805.0,-2.31,-6.37,44.91,35.67,5.0,-0.15,2.13,6.60,X40N,ATH,FMCG
39,INFY,2275.00,-18.16,49.0,H-LC,8.19,318840.0,5952.0,165733.0,-2.81,1.90,51.98,54.87,3.0,0.04,2.29,8.68,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,1.03,55.0,H-LC,3.59,162250.0,-23845.0,71325.0,-2.43,-12.81,43.96,25.51,15.0,-0.33,1.17,19.57,AR,ATH,ELECTRICAL
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198974.0,-1164.0,22046.0,-0.91,-0.58,11.08,10.44,4.0,-0.05,1.43,4.95,X40,NTT,FMCG
19,CIPLA,1795.00,-20.81,45.0,H-LC,5.49,211336.0,6836.0,34575.0,-2.11,3.34,16.36,20.25,10.0,0.20,1.52,12.37,X40N,BTT,PHARMA
56,RELIANCE,1533.00,-13.29,49.0,H-LC,4.80,216949.0,6583.0,22194.0,-1.19,3.13,10.23,13.68,37.0,0.30,1.56,20.17,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-13.45,45.0,H-LC,7.73,218539.0,-33229.0,86432.0,0.07,-13.20,39.55,21.13,27.0,-0.38,1.57,17.55,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-27.40,44.0,H-LC,12.24,288928.0,-57028.0,131896.0,-3.01,-16.48,45.65,21.64,1.0,-0.43,2.08,2.35,X40,ATH,IT
39,INFY,2275.00,-18.16,49.0,H-LC,8.19,318840.0,5952.0,165733.0,-2.81,1.90,51.98,54.87,3.0,0.04,2.29,8.68,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,H-LC,2.28,198974.0,-1164.0,22046.0,-0.91,-0.58,11.08,10.44,4.0,-0.05,1.43,4.95,X40,NTT,FMCG
82,VBL,671.64,-17.93,37.0,H-LC,6.88,295713.0,-20129.0,132805.0,-2.31,-6.37,44.91,35.67,5.0,-0.15,2.13,6.60,X40N,ATH,FMCG
1,ABB,7934.00,-38.31,60.0,H-LC,9.43,257784.0,-3835.0,123040.0,-1.22,-1.47,47.73,45.57,7.0,-0.03,1.85,9.69,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7094.44,68.0,L-SC,5.17,78919.0,-14891.0,91507.0,0.78,-15.87,115.95,81.67,269.0,-0.16,0.57,52.83,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.28,82656.0,-30893.0,70944.0,-2.87,-27.21,85.83,35.27,268.0,-0.44,0.59,102.30,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,65.0,M-SC,3.91,84708.0,-17502.0,110891.0,-1.42,-17.12,130.91,91.37,208.0,-0.16,0.61,37.51,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-5.85,43.0,H-SC,12.29,87600.0,-13363.0,107897.0,-1.96,-13.24,123.17,93.63,148.0,-0.12,0.63,30.48,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.62,94500.0,-3480.0,25080.0,-0.11,-3.55,26.54,22.05,152.0,-0.14,0.68,37.36,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1255.66,80.0,M-SC,6.41,184435.0,3292.0,42180.0,10.63,1.82,22.87,25.10,235.0,0.08,1.33,47.81,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.81,71.0,H-SC,13.41,225262.0,-45499.0,375444.0,-2.33,-16.80,166.67,121.86,133.0,-0.12,1.62,76.22,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,65.0,M-SC,3.91,84708.0,-17502.0,110891.0,-1.42,-17.12,130.91,91.37,208.0,-0.16,0.61,37.51,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.00,63.64,66.0,H-SC,6.52,141857.0,-40918.0,151986.0,1.69,-22.39,107.14,60.77,125.0,-0.27,1.02,27.06,XR,NTT,JEWELLERY
49,NATIONALUM,244.55,-44.49,63.0,H-MC,8.54,111496.0,10992.0,18118.0,-0.58,10.94,16.25,28.96,79.0,0.61,0.80,48.62,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.59
1,25,44.17
2,50,75.30


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.81
LC    31.13
MC    24.03
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.89
X40      15.01
X40N     11.64
XR       11.48
AR        8.53
XY25      8.39
OX40N     7.98
X200      1.70
MH        1.67
X5K       1.44
SR        1.24
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.89
H-LC    24.56
H-MC    21.06
M-SC    13.39
M-LC     5.52
M-MC     2.67
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.19,-14.61,77.10
FMCG,12.42,-4.07,41.30
FINANCE,10.21,-15.51,65.90
MISC,7.17,-15.12,79.12
BANKS,6.19,-15.92,76.90
PAINTS,5.74,-15.56,32.79
ELECTRICAL,5.60,-5.83,42.97
INSURANCE,3.76,-5.98,45.59
AC,3.63,4.03,12.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3084794.0,22
XR,1314340.0,14
AR,1143846.0,9
X40,762354.0,10
X40N,587696.0,9
OX40N,537331.0,10
XY25,380884.0,6
SR,248795.0,2
X5K,117867.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3635798.0,29
M-SC,1383588.0,17
H-MC,1178890.0,15
H-LC,1112858.0,14
M-LC,383227.0,4
M-MC,306670.0,2
L-SC,248009.0,3
L-MC,60719.0,1
L-LC,38018.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1219659.0      6
M-SC       XY24       792733.0      7
H-SC       AR         784815.0      5
           XR         765415.0      7
H-MC       XY24       542503.0      4
H-LC       X40        480912.0      5
M-MC       XY24       306670.0      2
H-SC       X40N       296510.0      4
           OX40N      257411.0      4
           SR         248795.0      2
H-LC       X40N       219431.0      3
H-MC       X40        209049.0      4
H-LC       AR         194365.0      2
H-MC       XY25       165432.0      2
M-SC       AR         164666.0      2
L-SC       XR         162451.0      2
M-LC       XY24       160181.0      2
M-SC       OX40N      129027.0      4
M-LC       X5K        117867.0      1
M-SC       XR         113878.0      2
           XY25       110891.0      1
H-MC       XR         106698.0      1
M-LC       XR         105179.0      1
H-LC       X200        88559.0      1
L-SC       OX40N       85558.0      1
M-SC       X40         72393.0      1
H-MC       X40N        71755.0      2
H-LC       XY25        66543.0      2
H-MC       OX40N       65335.0      1
H-SC       MH          63193.0      1
H-LC       XY24        63048.0      1
L-MC       XR          60719.0      1
L-LC       XY25        38018.0      1
H-MC       MH          18118.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
